# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [40]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [41]:
sms_spam_df = spark.read.csv('/datasets/sms_spam.csv', header=True, inferSchema=True)

# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`

In [42]:
# create sms_spam2_df below
# YOUR CODE HERE

#Making a new dataframe with text column and a corresponding dummy column for type, with spam=1 else 0:
sms_spam2_df=sms_spam_df.select(fn.when(fn.col("type")=="spam",1).otherwise(0).alias("type"),fn.col("text"))

#Checking to see if results are correct:
sms_spam_df.show(5)
sms_spam2_df.show(5)

#Looking at the number of rows:
sms_spam2_df.count()

#raise NotImplementedError()

+----+--------------------+
|type|                text|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



5574

In [43]:
sms_spam2_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [ ]:
sms_spam2_df = sms_spam_df.\
    select(
fn.when(fn.col('type')=='spam',1).otherwise(0).alias('type'),
fn.col('text')
)

In [ ]:
sms_spam2_df.show()

In [ ]:
sms_spam2_df.count()

In [44]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747]
)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [65]:
# create a Pipeline transformer and name it tfidf_pipeline
# YOUR CODE HERE

from pyspark.ml.feature import Tokenizer

#Setting up the toeknizer:
tokenizer=Tokenizer().setInputCol("text").setOutputCol("words")
#df1=tokenizer.transform(sms_spam2_df)

#Requesting word list from the URL:
import requests
stop_word_list=requests.get("http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words").text.split()

#Setting up a stop word remover:
from pyspark.ml.feature import StopWordsRemover
swr=StopWordsRemover().setStopWords(stop_word_list).setCaseSensitive(False).setInputCol("words").setOutputCol("filtered_words")

#Setting up a count vectorizer:
from pyspark.ml.feature import CountVectorizer

vectorizer_estimator=CountVectorizer().setInputCol("words").setOutputCol("tf")
#vectorizer_transformer=vectorizer_estimator.fit(df1)
#vectorizer_transformer.transform(df1).show()

#Setting up the IDF estimator:
from pyspark.ml.feature import IDF
idf=IDF().setInputCol("tf").setOutputCol("tfidf")

#Making a pipeline with all the above estimators as stages:
from pyspark.ml import Pipeline
tfidf_pipeline=Pipeline(stages=[tokenizer,vectorizer_estimator,idf]).fit(sms_spam2_df)
tfidf_pipeline.transform(sms_spam2_df).show(5)
#big_df=tfidf_pipeline.transform(sms_spam2_df)

#Made a stopword transformer, but it wasn't required as per the test case.
#raise NotImplementedError()

+----+--------------------+--------------------+--------------------+--------------------+
|type|                text|               words|                  tf|               tfidf|
+----+--------------------+--------------------+--------------------+--------------------+
|   0|Go until jurong p...|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|
|   0|Ok lar... Joking ...|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|
|   1|Free entry in 2 a...|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|
|   0|U dun say so earl...|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|
|   0|Nah I don't think...|[nah, i, don't, t...|(13525,[0,1,66,86...|(13525,[0,1,66,86...|
+----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



##### tfidf_pipeline.transform(sms_spam2_df).show()

In [46]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [47]:
# create sms_spam3_df below
# YOUR CODE HERE

#Checking for substrings with length of 3 or more than 3, and if found, dummy variable is set to 1 for having 3 or more uppercase letters in a succession:
sms_spam3_df=sms_spam2_df.withColumn("has_uppercase",fn.when(fn.length(fn.regexp_extract(fn.col("text"),"[A-Z]{3,}",0))>2,1).otherwise(0))

#Checking if the regex was applied correctly or not:
sms_spam3_df.show(5)

#raise NotImplementedError()

+----+--------------------+-------------+
|type|                text|has_uppercase|
+----+--------------------+-------------+
|   0|Go until jurong p...|            0|
|   0|Ok lar... Joking ...|            0|
|   1|Free entry in 2 a...|            0|
|   0|U dun say so earl...|            0|
|   0|Nah I don't think...|            0|
+----+--------------------+-------------+
only showing top 5 rows



The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [48]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]

In [49]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [50]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [51]:
[training_df.count(), validation_df.count(), testing_df.count()]

[3349, 1674, 551]

**(5 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$). Fit those pipelines to the appropriate data split.

1. Logistic regression with $\lambda=0$ and $\alpha=0$ (assign the fitted pipeline to `lr_pipeline1`)
2. Logistic regression with $\lambda=0.02$ and $\alpha=0.2$ (assign the fitted pipeline to `lr_pipeline2`)
3. Logistic regression with $\lambda=0.1$ and $\alpha=0.4$ (assign the fitted pipeline to `lr_pipeline3`)

In [52]:
# create lr_pipeline1, lr_pipeline2, and lr_pipeline3
# YOUR CODE HERE

#First made 3 estimators for 3 different cases of logistic regression and then individually fit them on training data and used the resultant transformer of the validation data:
from pyspark.ml.classification import LogisticRegression
lr1=LogisticRegression().setLabelCol("type").setFeaturesCol("tfidf").setRegParam(0).setMaxIter(100).setElasticNetParam(0)
lr2=LogisticRegression().setLabelCol("type").setFeaturesCol("tfidf").setRegParam(0.02).setMaxIter(100).setElasticNetParam(0.2)
lr3=LogisticRegression().setLabelCol("type").setFeaturesCol("tfidf").setRegParam(0.1).setMaxIter(100).setElasticNetParam(0.4)

lr_pipeline1=Pipeline(stages=[tfidf_pipeline,lr1]).fit(training_df)
lr_pipeline1.transform(validation_df)

lr_pipeline2=Pipeline(stages=[tfidf_pipeline,lr2]).fit(training_df)
lr_pipeline2.transform(validation_df)

lr_pipeline3=Pipeline(stages=[tfidf_pipeline,lr3]).fit(training_df)
lr_pipeline3.transform(validation_df)

#Taking a look at the validation accuracy for every model:
lr_pipeline1.transform(validation_df).select(fn.avg(fn.expr('float(prediction = type)')).alias("Validation accuracy for model 1")).show()
lr_pipeline2.transform(validation_df).select(fn.avg(fn.expr('float(prediction = type)')).alias("Validation accuracy for model 2")).show()
lr_pipeline3.transform(validation_df).select(fn.avg(fn.expr('float(prediction = type)')).alias("Validation accuracy for model 3")).show()
#raise NotImplementedError()

+-------------------------------+
|Validation accuracy for model 1|
+-------------------------------+
|             0.9659498207885304|
+-------------------------------+

+-------------------------------+
|Validation accuracy for model 2|
+-------------------------------+
|             0.9683393070489845|
+-------------------------------+

+-------------------------------+
|Validation accuracy for model 3|
+-------------------------------+
|             0.9080047789725209|
+-------------------------------+



In [53]:
# (10 pts)
np.testing.assert_equal(type(lr_pipeline1), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline2), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline3), pipeline.PipelineModel)
np.testing.assert_array_equal([type(s) for s in lr_pipeline1.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline2.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline3.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

**(5 pts)** Use the evaluator object defined below to compute the area under the curve of your predictors. For example, to compute the area under the curve of pipeline 1 for a dataframe `df`, you would run

```python
evaluator.evaluate(lr_pipeline1.transform(df))
```

Assign the AUC of the three models to the variables `AUC1`, `AUC2`, and `AUC3`, and and assign the pipeline with the best model to a variable `best_model`

In [54]:
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='type')

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [55]:
# print the AUC for the three models as follows
# print("Model 1 AUC: ", evaluator.evaluate(....))
# etc
# finally, based on these, assign the best validated 
# model to a variable best_model
# YOUR CODE HERE

#AUC for every model created:
AUC1=evaluator.evaluate(lr_pipeline1.transform(validation_df))
AUC2=evaluator.evaluate(lr_pipeline2.transform(validation_df))
AUC3=evaluator.evaluate(lr_pipeline3.transform(validation_df))

print("AUC for Model 1: ",AUC1)
print("AUC for Model 2: ",AUC2)
print("AUC for Model 3: ",AUC3)

#The model from lr_pipeline2 has the highest AUC and thus assigend as best model:
best_model=lr_pipeline2

#raise NotImplementedError()

AUC for Model 1:  0.9557218489415972
AUC for Model 2:  0.9871555611031791
AUC for Model 3:  0.9686667539402503


In [56]:
# (5 pts)
np.testing.assert_array_equal([type(AUC1), type(AUC2), type(AUC3)],
                             [float, float, float])
# AUC less than 1
np.testing.assert_array_less([AUC1, AUC2, AUC3], [1, 1, 1])
# AUC more than 0.5
np.testing.assert_array_less([.5, .5, .5],
                            [AUC1, AUC2, AUC3])

# Question 2.5: Choose best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [57]:
# assign to AUC_best the AUC of the best model selected before
# YOUR CODE HERE

#The AUC for the best model is:
AUC_best=evaluator.evaluate(best_model.transform(testing_df))
print(AUC_best)

#raise NotImplementedError()

0.976126746201693


In [58]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 
                               0.976126746201693, significant=2)

# Question 2.6: Inference

Use the pipeline 2 fitted above (`lr_pipeline2`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [59]:
# create positive_words and negative_words pandas dataframe below
# YOUR CODE HERE

#Since I have a pipeline inside a pipeline, I am accessing inner pipeline (lr_pipeline2) for the vocabulary (from IDF estimator):
inner_pipeline=lr_pipeline2.stages[0]
#print(type(inner_pipeline))

#Taking the weights from the lr_pipeline2's last stage, which is the actual logistic regression (thus taking the coefficients):
pipeline_word_weights=lr_pipeline2.stages[-1].coefficients.toArray()

#Positive words list (descending order to have highest weights up):
positive_words=pd.DataFrame({"word":inner_pipeline.stages[1].vocabulary,"weight":pipeline_word_weights}).sort_values("weight",ascending=False).head(20)

#Negative words list:
negative_words=pd.DataFrame({"word":inner_pipeline.stages[1].vocabulary,"weight":pipeline_word_weights}).sort_values("weight").head(20)

#raise NotImplementedError()

In [60]:
# examine positive vocabulary
positive_words.head()

,word,weight
3555,widelive.com/index.,0.590870
12237,08714712388,0.533567
15,call,0.517100
81,txt,0.513278
9064,gbp/sms,0.468274


In [61]:
# examine solutions
negative_words.head()

,word,weight
1,i,-0.162493
2444,fighting,-0.060939
3221,dificult,-0.059061
3371,fightng,-0.059061
3332,lose.,-0.059061


The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.590870</td>
    </tr>
    <tr>
      <th>12237</th>
      <td>08714712388</td>
      <td>0.533567</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.517100</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.513278</td>
    </tr>
    <tr>
      <th>9064</th>
      <td>gbp/sms</td>
      <td>0.468274</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.162493</td>
    </tr>
    <tr>
      <th>2444</th>
      <td>fighting</td>
      <td>-0.060939</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.059061</td>
    </tr>
  </tbody>
</table>

In [62]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.3701485692317927, significant=2)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.6661952507442954, significant=2)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

# Question 2.7
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [81]:
# your code and comments below
# YOUR CODE HERE

from pyspark.ml.feature import MaxAbsScaler

#Pasing sms_spam3_df (one with has_uppercase) through the tfidf_pipeline to get the tf-idf measure: 
tfidf_df=tfidf_pipeline.transform(sms_spam3_df)
#Creating a scalar to be applied:
scaler=MaxAbsScaler(inputCol="tfidf",outputCol="scaled_tfidf")
#Fitting the scalar on the data from previous step:
scaler_transformer=scaler.fit(tfidf_df)
#Transformer applied to the same data to do scaling:
scaled_df=scaler_transformer.transform(tfidf_df)

#Just converting to pandas to have a look if scaling is done correctly:
sms_spam3_df_scaled=scaled_df.toPandas()
sms_spam3_df_scaled.head()

#Collapsing has_uppercase and newly scaled tfidf values to one features column:
vec_as=feature.VectorAssembler(inputCols=["has_uppercase","scaled_tfidf"],outputCol="features")
for_log_reg_df=vec_as.transform(scaled_df)
for_log_reg_df.show(5)

#raise NotImplementedError()

+----+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|type|                text|has_uppercase|               words|                  tf|               tfidf|        scaled_tfidf|            features|
+----+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   0|Go until jurong p...|            0|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|(13526,[9,43,52,6...|
|   0|Ok lar... Joking ...|            0|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|(13525,[5,74,404,...|(13526,[6,75,405,...|
|   1|Free entry in 2 a...|            0|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|(13526,[1,4,9,21,...|
|   0|U dun say so earl...|            0|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|(13525,[5,22,6

In [82]:
#Making a new pipeling for the vectorized data with lambda=0.2 and alpha=0.1, fitting the pipeline on all of the data and creating a model:
lr_pipeline=Pipeline(stages=[LogisticRegression().setLabelCol("type").setFeaturesCol("features").setRegParam(0.2).setMaxIter(100).setElasticNetParam(0.1)])
lr_model=lr_pipeline.fit(for_log_reg_df)

In [88]:
#Doing the same thing as I did for positive and negative words data frame:
#Used the vocabulary from lr_pipeline2:
inner_pipeline=lr_pipeline2.stages[0]
word_list=inner_pipeline.stages[1].vocabulary
#Added 'has_uppercase' at the start, for the first coefficient will correspond to it:
word_list.insert(0,"has_uppercase")
#Taking the coefficients:
coeff_list=lr_model.stages[0].coefficients.toArray()
#Making a dataframe out of it:
coeff_df=pd.DataFrame({"word":word_list,"coefficient":coeff_list})
coeff_df.head(5)

,word,coefficient
0,has_uppercase,0.928918
1,to,0.917446
2,i,-0.737638
3,you,0.000000
4,a,0.153977


In [91]:
#Coefficient for has_uppercase:
has_uppercase_coeff=coeff_df.loc[0,"coefficient"]
print("Coefficient for 'has_uppercase': ",has_uppercase_coeff)

Coefficient for 'has_uppercase':  0.9289178747599827


In [97]:
#Took all coefficients in the list, removed the coefficient for has_uppercase, and then sorted it in a descending fashion. Eventually pulled the biggest positive coeefficient:
coeff_val=list(coeff_df.coefficient)
coeff_val=coeff_val[1:]
largest_coeff_word=sorted(coeff_val,reverse=True)[0]
ratio=has_uppercase_coeff/largest_coeff_word
print("The ratio between 'has_uppercase' coefficient and the biggest positive tfidf coefficient is: ",ratio)

The ratio between 'has_uppercase' coefficient and the biggest positive tfidf coefficient is:  0.4617005798337622
